In [3]:
from torrent.torrent_file import generate_torrent_file, read_file, decode_raw_data, get_trackers, create_tracker
from urllib.parse import urlparse
from torrent.tracker import request_manager, http_request, create_url_for_http_tracker, parse_tracker_res
from torrent.bencoding import Decoder, Encoder
from time import sleep
from random import randint, choice
import logging
import struct
import string
from torrent.utilities import run_async
import socket

path = '../test_files/file3.torrent'
logger = logging.getLogger(__name__)
BLOCK_SIZE = 2 ** 11


In [2]:
torrent_file = generate_torrent_file(path)
l = []
logger.info("http req start")
for tracker in torrent_file.trackers:
    url = create_request_dict(torrent_file, tracker, torrent_file.length)
    l.append(http_request(url))
    
logger.info("http queue create")
while True:
    out = False
    for t in l:
        if t.isAlive() == False:
            res = t.result_queue.get()
            peers = parse_tracker_res(res)
            out = True
    if out:
        break
    sleep(0.1)
logger.info("peer and res loaded")


2020-03-24 17:50:28,284 - __main__ - INFO - 3 http req start ;
2020-03-24 17:50:28,300 - __main__ - INFO - 8 http queue create ;
2020-03-24 17:50:29,507 - __main__ - INFO - 19 peer and res loaded ;


In [3]:
def open_socket_tcp(peer_data):
    my_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        my_socket.connect(peer_data)
    except Exception as e:
        logger.info("connect error", exc_info=True)
        return None
    return my_socket

In [4]:
def handshake(torrent_file):
    const_protocol = 'BitTorrent protocol'.encode('utf-8')
    info_hash = torrent_file.info_hash
    peer_id = torrent_file.id.encode('utf-8')
    return struct.pack('>B19s8x20s20s', 19, const_protocol, info_hash, peer_id)

def keep_alive():
    return b'0000'

def interested_message():
    return struct.pack('>IB', 1, 2) 

def request_message(piece_index, block_offset, block_size):
    return struct.pack('>IBIII', 13, 6, piece_index, block_offset, block_size)



In [5]:
def parse_message(res, message_id_expected):
    # -1 is handshaking
    # 
    # 5 is bitfield 
    
    
    if message_id_expected != 7:
        logger.info(res)
    if message_id_expected == -1:
        if len(res) != 68:
            logger.info('you didnt got handshake')
            return False
        else:
            return True
    
    if message_id_expected == 1:
        if res == b'':
            logger.info('intersted message answered with nothing')
            return False
        elif res[4] == 0:
            logger.info('the peer choke')
            return False
        elif res[4] == 1:
            return True
        else:
            logger.info(f"somthing got worng with intersted message - '{res}' got from peer")
            return False
              
    if message_id_expected == 5:
        if res == b'':
            logger.info('peer not hold any piece from file')
            return False
        if res[4] != 5:
            return False
        return True
    
    if message_id_expected == 7:
        if res == b'':
            logger.info('peer request failed and got empty response')
            return False
        elif res[4] != 7: 
            logger.info(f"the message id is not 7 but you got - '{res[4]}'")
            return False
        logger.info("response got ok")
        return res
            
    
    
    
            

In [6]:
@run_async
def peer_manager(torrent_file, peer_id):
    #Conncet 
    peer_socket = open_socket_tcp(peer_id)
    if not peer_socket:
        logger.info('connect peer fail')
        return False
    
    logger.info("connection succeed")
    #Handshake
    peer_socket.send(handshake(torrent_file))
    res = parse_message(peer_socket.recv(68), -1 )
    if not res:
        logger.info('handshake peer failed')
        return False
    logger.info('handshake succeeded')
    
    #Bitfield
    sleep(0.1)
    res = parse_message(peer_socket.recv(2048), 5)
    if not res:
        logger.info('bitfield failed')
        return False
    logger.info('bitfield succeeded')
    
    #Interested
    peer_socket.send(interested_message())
    res = parse_message(peer_socket.recv(5), 1)
    if not res:
        logger.info('peer choke or got nothing')
        return False
    logger.info('peer unchoked')
    
#Request Piece
    offset = 0
    block_data = []
    while torrent_file.piece_length >= offset: 
        peer_socket.send(request_message(1, offset, BLOCK_SIZE))
        res = parse_message(peer_socket.recv(int(BLOCK_SIZE *1.12)), 7)
        if not res:
            logger.info("request Piece failed")
            return block_data
        else:
            block_data.append(res)
            offset += BLOCK_SIZE
    
    logger.info("all piece is here")
    return block_data


# orgnize test

In [7]:
def handshake(torrent_file):
    const_protocol = 'BitTorrent protocol'.encode('utf-8')
    info_hash = torrent_file.info_hash
    peer_id = torrent_file.id.encode('utf-8')
    return struct.pack('>B19s8x20s20s', 19, const_protocol, info_hash, peer_id)

def keep_alive():
    return b'0000'

def interested_message():
    return struct.pack('>IB', 1, 2) 

def request_message(piece_index, block_offset, block_size):
    return struct.pack('>IBIII', 13, 6, piece_index, block_offset, block_size)

In [9]:
def get_message_type(res):
    """
        False empty 
        -1 - keep alive
        0 - choke
        1 - unchoke
        2 - interested
        3 - uninterested
        4 - have
        5 - bitfield (what i got my brother)
        6 - request
        7 - piece
        8 - cancel
    """

    #empty
    if res == b'':
        logger.info('we got empty response')
        return None, None
    #keep alive
    elif len(res) == 4 and res == b'0000':
        return -1, 0
    #message type
    elif res[4] in range(0,8):
        parsed_res = struct.unpack(">IB", res)
        if not parse_message:
            logger.info("parsed_res got broken")
            return None, None
        return parsed_res
    else:
        logger.info(f"we got for response {res[4]}")
        return None, None



def valid_res_hand_shake(res):
    if len(res) != 68:
        logger.info('you didnt got handshake')
        return False
    else:
        return True

1024
1024
1024
1024

In [10]:
#@run_async
def peer_manager(torrent_file, peer_id):
    #Conncet 
    peer_socket = open_socket_tcp(peer_id)
    if not peer_socket:
        logger.info('connect peer fail')
        return False
    logger.info("connection succeed")
    
    #Handshake
    peer_socket.send(handshake(torrent_file))
    res = valid_res_hand_shake(peer_socket.recv(68))
    if not res:
        logger.info('handshake peer failed')
        return False
    logger.info('handshake succeeded')
    
    #Bitfield
    size, message_type = get_message_type(peer_socket.recv(5))
    if message_type == 5:
        bitfiled_data = peer_socket.recv(size + 10)
        if len(bitfiled_data) + 1 != size:
            #TODO handle check what pieces peer has
            logger.info(f"bitfiled size is {size} and we got {len(bitfiled_data)}")
    else:
        logger.info(f"in bitfield we got '{message_type}' insted of 5")
    
    #Interested
    peer_socket.send(interested_message())
    size, message_type = get_message_type(peer_socket.recv(5))
    if message_type == 0:
        logger.info('peer choke')
        return False
    elif message_type == None:
        return False
    elif message_type == 1:
        logger.info('peer unchoke')
    
    #Request Piece
    offset = 0
    chunks = []
    counter = 0
    while torrent_file.piece_length >= offset:
        #request 
        peer_socket.send(request_message(7, offset, BLOCK_SIZE))
        meta_message = peer_socket.recv(5)
        if not meta_message:
            logger.info(f"we got nothing from request in rount {counter}")
            break
        size, message_type = get_message_type(meta_message) 
        if message_type != 7:
            logger.info(f"in request we got '{message_type}' insted of 7")
            break
        if not size or not message_type:
            logger.info(f"somthing get worng in {counter} request {meta_message} got from peer")
            
        #chunks
        res = peer_socket.recv(size)
        if res == b'':
            logger.info("we got empty data")
            break
            
        chunk_index, chunk_offset = struct.unpack(f">II", res[0:8])
        offset += BLOCK_SIZE
        chunks.append(res[8:len(res)])
        
                    
        logger.info(f"request {counter} get ok")
        counter += 1
    return chunks
  

        


    


In [40]:
peer = peers[5]

In [41]:
data = peer_manager(torrent_file, peer)

KeyboardInterrupt: 

In [29]:
piece = b''
for c in data:
    piece += c
    

In [27]:
from hashlib import sha1

In [31]:
s1 = sha1(piece)

b'\xd7\xb0\x05\x17\xc6\x91~\xd9\xcb5b\x8f\xd8/\x89sO\xbc\x1c\xe8'

In [36]:
torrent_file.pieces[7*20:8*20]

b'\xd7\xb0\x05\x17\xc6\x91~\xd9\xcb5b\x8f\xd8/\x89sO\xbc\x1c\xe8'

In [42]:
peers

[('190.188.48.59', 51413),
 ('68.230.192.50', 54536),
 ('98.5.16.14', 54194),
 ('86.236.166.216', 16881),
 ('94.180.82.193', 51413),
 ('82.125.24.243', 51413),
 ('162.157.107.223', 23251),
 ('201.231.117.248', 51413),
 ('73.243.164.12', 51413),
 ('83.85.194.25', 16881),
 ('151.80.168.186', 51413),
 ('195.242.156.241', 1194),
 ('5.79.77.65', 50670),
 ('188.226.31.242', 51413),
 ('185.203.56.10', 55581),
 ('86.202.20.139', 16881),
 ('45.142.213.64', 38675),
 ('81.249.235.83', 27376),
 ('188.213.31.37', 51413),
 ('185.21.217.58', 58090),
 ('5.230.148.115', 8999),
 ('67.223.21.157', 51413),
 ('78.218.213.18', 60626),
 ('151.29.29.214', 51413),
 ('165.22.115.234', 51413)]